In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [2]:
teams_concat = pd.read_csv('2018master.csv') # read in training set

In [3]:
teams_concat_test = pd.read_csv('2019master.csv') # read in test set

In [4]:
# divide training set into predictors/ responses
X_train = teams_concat.drop(["date", 'Opposing Team', 'Opposing Starter', 'Opposing Team Score',
                      'Won','Runs Scored'], 1)

y_train_reg = teams_concat['Runs Scored']
y_train_cl = teams_concat['Won']

In [5]:
# divide test set into predictors/responses
X_test = teams_concat_test.drop(["Team", "date", 'Opposing Team', 'Opposing Starter', 'Opposing Team Score',
                      'Won','Runs Scored'], 1)
y_test_reg = teams_concat_test['Runs Scored']
y_test_cl = teams_concat_test["Won"]

## Standardization to perform PCA

In [26]:
# want to do it on the scaled data set
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_scaled = scaler.transform(X_train)

from sklearn.decomposition import PCA
pca = PCA()
pca_data = pca.fit_transform(X_scaled)
pca_data_df = pd.DataFrame(pca_data)

var_exp = pca.explained_variance_ratio_
print("Variance in each PC:", var_exp) 

Variance in each PC: [3.19463866e-01 9.63268787e-02 7.46201155e-02 6.40509923e-02
 4.52214858e-02 4.50956068e-02 3.88045521e-02 3.32196487e-02
 2.97387695e-02 2.82373116e-02 2.63409699e-02 2.54123800e-02
 2.42966740e-02 2.42029234e-02 2.02622032e-02 1.93730565e-02
 1.59165422e-02 1.39463116e-02 1.27015518e-02 1.10029511e-02
 7.69954565e-03 5.57751057e-03 5.24920049e-03 3.24578213e-03
 2.75310755e-03 2.37309431e-03 2.27159985e-03 1.33391157e-03
 1.03888268e-03 1.58992185e-04 5.34878333e-05 5.48109328e-06
 2.42738691e-06 1.03619630e-06 8.43401062e-07 2.93990810e-07
 1.29166543e-08 6.30031386e-32]


In [7]:
cum_var_exp[8]

0.7465419149289534

## K-fold CV for decision trees

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor()

params = {"max_depth": list(range(1,41,3)), "max_features":list(range(1,39,3))}

grid_search = GridSearchCV(dtr, params, cv=10, scoring='neg_mean_squared_error', verbose = 1)

grid_search.fit(X_train, y_train_reg)

print(grid_search.best_params_)
print("RMSE: ", grid_search.best_score_)

Fitting 10 folds for each of 182 candidates, totalling 1820 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


{'max_depth': 4, 'max_features': 28}
RMSE:  -9.825520535694864


[Parallel(n_jobs=1)]: Done 1820 out of 1820 | elapsed:   45.5s finished


In [17]:
dtr = DecisionTreeRegressor()

params = {"max_depth": list(range(1,10)), "max_features":list(range(23,34))}

# Create the GridSearchCV object:
grid_search = GridSearchCV(dtr, params, cv=100, scoring='neg_mean_squared_error', verbose = 1)

grid_search.fit(X_train, y_train_reg)

print(grid_search.best_params_)
print("RMSE: ", grid_search.best_score_)

Fitting 100 folds for each of 99 candidates, totalling 9900 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


{'max_depth': 3, 'max_features': 33}
RMSE:  -9.743448167582686


[Parallel(n_jobs=1)]: Done 9900 out of 9900 | elapsed:  3.0min finished
C:\Users\jacob\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [19]:
dtr = DecisionTreeRegressor()

params = {"max_depth": list(range(1,5)), "max_features":list(range(30,39))}

# Create the GridSearchCV object:
grid_search = GridSearchCV(dtr, params, cv=1000, scoring='neg_mean_squared_error', verbose = 1)

grid_search.fit(X_train, y_train_reg)

print(grid_search.best_params_)
print("RMSE: ", grid_search.best_score_)

Fitting 1000 folds for each of 36 candidates, totalling 36000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


{'max_depth': 3, 'max_features': 31}
RMSE:  -9.780283395836287


[Parallel(n_jobs=1)]: Done 36000 out of 36000 | elapsed:  9.9min finished
C:\Users\jacob\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


# Cross Validation For SVM

In [8]:
import warnings
warnings.simplefilter("ignore")
from sklearn.svm import SVC

In [21]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
scaler = StandardScaler()

pca = PCA()

svc = SVC()

pipe = Pipeline(steps=[('scaler', scaler), ('pca', pca), ('SVM', svc)])

param_grid = {"SVM__kernel":['linear', 'rbf', 'poly', 'sigmoid'], 
              "SVM__C":list(range(40,101,20)), "pca__n_components":[23,25,27]}

grid_search = GridSearchCV(pipe, param_grid, cv = 3, scoring = 'accuracy', verbose = 1)
grid_search.fit(X_train, y_train_cl)
print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 144 out of 144 | elapsed: 24.4min finished


0.5740436034553682
{'SVM__C': 80, 'SVM__kernel': 'linear', 'pca__n_components': 23}


In [9]:
pca = PCA()

svc = SVC()

pipe = Pipeline(steps=[('scaler', scaler), ('pca', pca), ('SVM', svc)])

param_grid = {"SVM__kernel":['linear', 'rbf','sigmoid'], 
              "SVM__C":list(range(70,100,10)), "pca__n_components":list(range(3,24,5))}

grid_search = GridSearchCV(pipe, param_grid, cv = 10, scoring = 'accuracy', verbose = 1)
grid_search.fit(X_train, y_train_cl)
print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 10 folds for each of 45 candidates, totalling 450 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 450 out of 450 | elapsed: 46.3min finished


0.6034553681612506
{'SVM__C': 90, 'SVM__kernel': 'sigmoid', 'pca__n_components': 8}


In [11]:
from sklearn.svm import SVC
pca = PCA()
scaler = StandardScaler()
svc = SVC()

pipe = Pipeline(steps=[('scaler', scaler), ('pca', pca), ('SVM', svc)])

param_grid = {"SVM__C":list(range(85,101,5)), "pca__n_components":list(range(7,10)), "SVM__kernel":["sigmoid", "linear"]}

grid_search = GridSearchCV(pipe, param_grid, cv = 10, scoring = 'accuracy', verbose = 1)
grid_search.fit(X_train, y_train_cl)
print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 10 folds for each of 24 candidates, totalling 240 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 240 out of 240 | elapsed: 21.9min finished


0.6013986013986014
{'SVM__C': 90, 'SVM__kernel': 'sigmoid', 'pca__n_components': 8}


## CV Loop with a tiny cost

In [13]:
pca = PCA()

svc = SVC(.01)

pipe = Pipeline(steps=[('scaler', scaler), ('pca', pca), ('SVM', svc)])

param_grid = {"pca__n_components":list(range(3,10,3)), "SVM__kernel":['sigmoid','linear','rbf']}

grid_search = GridSearchCV(pipe, param_grid, cv = 10, scoring = 'accuracy', verbose = 1)
grid_search.fit(X_train, y_train_cl)
print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 10 folds for each of 9 candidates, totalling 90 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:  2.4min finished


0.6217605923488276
{'SVM__kernel': 'sigmoid', 'pca__n_components': 9}


In [18]:
pca = PCA()
svc = SVC(.01)

pipe = Pipeline(steps=[('scaler', scaler), ('pca', pca), ('SVM', svc)])

param_grid = {"pca__n_components":list(range(6,13)), "SVM__kernel":['sigmoid','linear']}

grid_search = GridSearchCV(pipe, param_grid, cv = 30, scoring = 'accuracy', verbose = 1)
grid_search.fit(X_train, y_train_cl)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 30 folds for each of 14 candidates, totalling 420 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 420 out of 420 | elapsed: 13.8min finished


0.6213492389962978
{'SVM__kernel': 'sigmoid', 'pca__n_components': 6}


## CV loops to find ideal number of SVMs to ensemble

In [8]:
from sklearn.neural_network import MLPRegressor # multilayer perceptron

In [26]:
#create a PCA
pca = PCA(n_components = 8)

from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(base_estimator=SVC(kernel = "sigmoid", C = .01, probability = True),verbose = True)

#create a pipeline that does scaling, then PCA, then SVM
pipe = Pipeline(steps=[('scaler', scaler), ('pca', pca), ('BaggingClassifier', clf)])

#Set up the parameters you want to tune for each of your pipeline steps
#Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {"BaggingClassifier__n_estimators":list(range(100,1101, 200))}

# pass the pipeline and the parameters into a GridSearchCV with a 3-fold cross validation
grid_search = GridSearchCV(pipe, param_grid, cv = 3, scoring = 'accuracy', verbose = 3)
# call fit() on the GridSearchCV and pass in the unscaled data (X_values, Y_values)
grid_search.fit(X_train, y_train_cl)
# print out the best_score_ and best_params_ from the GridSearchCV
print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] BaggingClassifier__n_estimators=100 .............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min remaining:    0.0s
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] . BaggingClassifier__n_estimators=100, score=0.603, total= 1.5min
[CV] BaggingClassifier__n_estimators=100 .............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.9s finished
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.0min remaining:    0.0s
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] . BaggingClassifier__n_estimators=100, score=0.582, total= 1.5min
[CV] BaggingClassifier__n_estimators=100 .............................


...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 14.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  BaggingClassifier__n_estimators=1100, score=0.581, total=16.0min
[CV] BaggingClassifier__n_estimators=1100 ............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 14.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min finished
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed: 205.9min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  BaggingClassifier__n_estimators=1100, score=0.578, total=15.7min
0.5878239407651172
{'BaggingClassifier__n_estimators': 700}


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 23.1min finished


# Nested CV loop for neural net with one layer

In [7]:
from sklearn.neural_network import MLPRegressor

In [21]:
scaler = StandardScaler()
pca = PCA()

MLP = MLPRegressor(early_stopping = True, max_iter = 1000, learning_rate = 'adaptive')

step = [('scaler', scaler),('pca', pca),("MLPRegressor", MLP)]
pipe = Pipeline(step)
param_grid = {"MLPRegressor__hidden_layer_sizes":[(10,),(30,),(50,),(70,),(90,)], 
              'pca__n_components':list(range(1,32,5)), "MLPRegressor__activation":['identity','logistic', 'tanh', 'relu']}
grid_search = GridSearchCV(pipe, param_grid, cv = 10, verbose = 1, scoring = 'neg_mean_squared_error')

grid_search.fit(X_train, y_train_reg)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 10 folds for each of 140 candidates, totalling 1400 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1400 out of 1400 | elapsed: 27.6min finished


-9.51963933809116
{'MLPRegressor__activation': 'logistic', 'MLPRegressor__hidden_layer_sizes': (50,), 'pca__n_components': 6}


In [22]:
scaler = StandardScaler()
pca = PCA()
MLP = MLPRegressor(early_stopping = True, max_iter = 1000, learning_rate = 'adaptive')

step = [('scaler', scaler),('pca', pca),("MLPRegressor", MLP)]
pipe = Pipeline(step)
param_grid = {"MLPRegressor__hidden_layer_sizes":list(range(30,71,5)),
              'pca__n_components':list(range(1,11,2)), "MLPRegressor__activation":['identity','logistic', 'tanh', 'relu']}
grid_search = GridSearchCV(pipe, param_grid, cv = 10, verbose = 1, scoring = 'neg_mean_squared_error')

grid_search.fit(X_train, y_train_reg)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 10 folds for each of 180 candidates, totalling 1800 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1800 out of 1800 | elapsed: 33.5min finished


-9.512420666681672
{'MLPRegressor__activation': 'logistic', 'MLPRegressor__hidden_layer_sizes': 60, 'pca__n_components': 9}


In [8]:
scaler = StandardScaler()

pca = PCA()

MLP = MLPRegressor(early_stopping = True, max_iter = 1000, learning_rate = 'adaptive', activation = 'logistic')

step = [('scaler', scaler),('pca', pca),("MLPRegressor", MLP)]
pipe = Pipeline(step)
param_grid = {"MLPRegressor__hidden_layer_sizes":list(range(50,70)),
              'pca__n_components':list(range(7,12))}
grid_search = GridSearchCV(pipe, param_grid, cv = 20, verbose = 1, scoring = 'neg_mean_squared_error')

grid_search.fit(X_train, y_train_reg)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 20 folds for each of 100 candidates, totalling 2000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed: 32.2min finished


-9.517876072921288
{'MLPRegressor__hidden_layer_sizes': 54, 'pca__n_components': 8}


## Nested CV for a 2 layer net

In [25]:
widths = list(range(10,101,30))

tuple_list = []
for i in (range (0, len(widths))):
  elem1 = widths[i]
  for j in range(0, len(widths)):
    elem2 = widths[j]
    tuple_list.append((elem1, elem2))

In [10]:
scaler = StandardScaler()

pca = PCA()

MLP = MLPRegressor(early_stopping = True, max_iter = 1000, learning_rate = 'adaptive')

step = [('scaler', scaler),('pca', pca),("MLPRegressor", MLP)]
pipe = Pipeline(step)
param_grid = {"MLPRegressor__hidden_layer_sizes":tuple_list,
              'pca__n_components':list(range(3,24,5)), "MLPRegressor__activation":['identity','logistic', 'tanh', 'relu']}
grid_search = GridSearchCV(pipe, param_grid, cv = 10, verbose = 1, scoring = 'neg_mean_squared_error')

grid_search.fit(X_train, y_train_reg)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 10 folds for each of 320 candidates, totalling 3200 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 3200 out of 3200 | elapsed: 76.1min finished


-9.520054154016817
{'MLPRegressor__activation': 'identity', 'MLPRegressor__hidden_layer_sizes': (40, 100), 'pca__n_components': 8}


In [17]:
width1 = list(range(20,61,5))
width2 = list(range(80,121,5))
tuple_list = []
for i in (range (0, len(width1))):
  elem1 = width1[i]
  for j in range(0, len(width2)):
    elem2 = width2[j]
    tuple_list.append((elem1, elem2))

In [13]:
scaler = StandardScaler()

pca = PCA()

MLP = MLPRegressor(early_stopping = True, max_iter = 1000, learning_rate = 'adaptive')

step = [('scaler', scaler),('pca', pca),("MLPRegressor", MLP)]
pipe = Pipeline(step)
param_grid = {"MLPRegressor__hidden_layer_sizes":tuple_list,
              'pca__n_components':list(range(4,13,2)), "MLPRegressor__activation":['identity','logistic', 'tanh', 'relu']}
grid_search = GridSearchCV(pipe, param_grid, cv = 10, verbose = 1, scoring = 'neg_mean_squared_error')

grid_search.fit(X_train, y_train_reg)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 10 folds for each of 1620 candidates, totalling 16200 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 16200 out of 16200 | elapsed: 366.8min finished


-9.518032194398229
{'MLPRegressor__activation': 'identity', 'MLPRegressor__hidden_layer_sizes': (25, 80), 'pca__n_components': 6}


In [18]:
width1 = list(range(15,36,5))
width2 = list(range(40,86,5))
tuple_list = []
for i in (range (0, len(width1))):
  elem1 = width1[i]
  for j in range(0, len(width2)):
    elem2 = width2[j]
    tuple_list.append((elem1, elem2))

In [16]:
scaler = StandardScaler()

pca = PCA()

MLP = MLPRegressor(activation = 'identity',early_stopping = True, max_iter = 1000, learning_rate = 'adaptive')

step = [('scaler', scaler),('pca', pca),("MLPRegressor", MLP)]
pipe = Pipeline(step)
param_grid = {"MLPRegressor__hidden_layer_sizes":tuple_list,
              'pca__n_components':[7,8,9,10]}
grid_search = GridSearchCV(pipe, param_grid, cv = 20, verbose = 1, scoring = 'neg_mean_squared_error')

grid_search.fit(X_train, y_train_reg)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 20 folds for each of 200 candidates, totalling 4000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 4000 out of 4000 | elapsed: 24.8min finished


-9.522704724069866
{'MLPRegressor__hidden_layer_sizes': (15, 85), 'pca__n_components': 7}


### No improvement in CV error. Stop validating

## CV Loop for a three layer net

In [20]:
width1 = list(range(30,91,30))
width2 = list(range(30,91,30))
width3 = list(range(30,91,30))
tuple_list = []
for i in (range (0, len(width1))):
  elem1 = width1[i]
  for j in range(0, len(width2)):
    elem2 = width2[j]
    for k in range(len(width3)):
      elem3 = width3[k]  
      tuple_list.append((elem1, elem2, elem3))

In [21]:
scaler = StandardScaler()

pca = PCA(9)

MLP = MLPRegressor(early_stopping = True, max_iter = 1000, learning_rate = 'adaptive')

step = [('scaler', scaler),('pca', pca),("MLPRegressor", MLP)]
pipe = Pipeline(step)
param_grid = {"MLPRegressor__hidden_layer_sizes":tuple_list, "MLPRegressor__activation":['identity','logistic', 'tanh', 'relu']}
grid_search = GridSearchCV(pipe, param_grid, cv = 3, verbose = 1, scoring = 'neg_mean_squared_error')

grid_search.fit(X_train, y_train_reg)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 324 out of 324 | elapsed:  7.7min finished


-9.51946177356567
{'MLPRegressor__activation': 'identity', 'MLPRegressor__hidden_layer_sizes': (60, 60, 60)}


In [10]:
width1 = list(range(40,81,20))
width2 = list(range(40,81,20))
width3 = list(range(40,81,20))
tuple_list = []
for i in (range (0, len(width1))):
  elem1 = width1[i]
  for j in range(0, len(width2)):
    elem2 = width2[j]
    for k in range(len(width3)):
      elem3 = width3[k]  
      tuple_list.append((elem1, elem2, elem3))

In [11]:
scaler = StandardScaler()

pca = PCA(9)

MLP = MLPRegressor(early_stopping = True, max_iter = 1000, learning_rate = 'adaptive')

step = [('scaler', scaler),('pca', pca),("MLPRegressor", MLP)]
pipe = Pipeline(step)
param_grid = {"MLPRegressor__hidden_layer_sizes":tuple_list, "MLPRegressor__activation":['identity','logistic']}
grid_search = GridSearchCV(pipe, param_grid, cv = 5, verbose = 1, scoring = 'neg_mean_squared_error')

grid_search.fit(X_train, y_train_reg)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 270 out of 270 | elapsed:  8.7min finished


-9.538895591900763
{'MLPRegressor__activation': 'identity', 'MLPRegressor__hidden_layer_sizes': (80, 40, 80)}


In [13]:
width1 = list(range(60,121,20))
width2 = list(range(10,71,20))
width3 = list(range(60,121,20))
tuple_list = []
for i in (range (0, len(width1))):
  elem1 = width1[i]
  for j in range(0, len(width2)):
    elem2 = width2[j]
    for k in range(len(width3)):
      elem3 = width3[k]  
      tuple_list.append((elem1, elem2, elem3))

In [14]:
scaler = StandardScaler()

pca = PCA(9)

MLP = MLPRegressor(early_stopping = True, max_iter = 1000, learning_rate = 'adaptive')

step = [('scaler', scaler),('pca', pca),("MLPRegressor", MLP)]
pipe = Pipeline(step)
param_grid = {"MLPRegressor__hidden_layer_sizes":tuple_list, "MLPRegressor__activation":['identity','logistic']}
grid_search = GridSearchCV(pipe, param_grid, cv = 5, verbose = 1, scoring = 'neg_mean_squared_error')

grid_search.fit(X_train, y_train_reg)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 5 folds for each of 128 candidates, totalling 640 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 640 out of 640 | elapsed: 19.9min finished


-9.524704806638795
{'MLPRegressor__activation': 'identity', 'MLPRegressor__hidden_layer_sizes': (60, 30, 120)}


In [15]:
width1 = list(range(30,91,10))
width2 = list(range(10,51,10))
width3 = list(range(100,140,10))
tuple_list = []
for i in (range (0, len(width1))):
  elem1 = width1[i]
  for j in range(0, len(width2)):
    elem2 = width2[j]
    for k in range(len(width3)):
      elem3 = width3[k]  
      tuple_list.append((elem1, elem2, elem3))

In [16]:
scaler = StandardScaler()

pca = PCA(9)

MLP = MLPRegressor(activation = 'identity', early_stopping = True, max_iter = 1000, learning_rate = 'adaptive')

step = [('scaler', scaler),('pca', pca),("MLPRegressor", MLP)]
pipe = Pipeline(step)
param_grid = {"MLPRegressor__hidden_layer_sizes":tuple_list}
grid_search = GridSearchCV(pipe, param_grid, cv = 5, verbose = 1, scoring = 'neg_mean_squared_error')

grid_search.fit(X_train, y_train_reg)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 5 folds for each of 140 candidates, totalling 700 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:  7.0min finished


-9.517989917879966
{'MLPRegressor__hidden_layer_sizes': (30, 30, 100)}


In [17]:
width1 = list(range(10,51,10))
width2 = list(range(10,51,10))
width3 = list(range(80,121,10))
tuple_list = []
for i in (range (0, len(width1))):
  elem1 = width1[i]
  for j in range(0, len(width2)):
    elem2 = width2[j]
    for k in range(len(width3)):
      elem3 = width3[k]  
      tuple_list.append((elem1, elem2, elem3))

In [18]:
scaler = StandardScaler()

pca = PCA(9)

MLP = MLPRegressor(activation = 'identity', early_stopping = True, max_iter = 1000, learning_rate = 'adaptive')

step = [('scaler', scaler),('pca', pca),("MLPRegressor", MLP)]
pipe = Pipeline(step)
param_grid = {"MLPRegressor__hidden_layer_sizes":tuple_list}
grid_search = GridSearchCV(pipe, param_grid, cv = 5, verbose = 1, scoring = 'neg_mean_squared_error')

grid_search.fit(X_train, y_train_reg)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 5 folds for each of 125 candidates, totalling 625 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 625 out of 625 | elapsed:  5.1min finished


-9.522853280848597
{'MLPRegressor__hidden_layer_sizes': (20, 30, 110)}


No increase in CV error; end tuning here.

## K- Nearest Neighbors CV Loops

In [19]:
from sklearn.neighbors import KNeighborsClassifier

scaler = StandardScaler()

pca = PCA()

knc = KNeighborsClassifier()

step = [('scaler', scaler),('pca', pca),("KNeighborsClassifier", knc)]
pipe = Pipeline(step)
param_grid = {"KNeighborsClassifier__n_neighbors":list(range(500,2001,500)), "pca__n_components":list(range(1,26,5))}
grid_search = GridSearchCV(pipe, param_grid, cv = 10, verbose = 1, scoring = 'accuracy')

grid_search.fit(X_train, y_train_cl)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.5619086795557384
{'KNeighborsClassifier__n_neighbors': 2000, 'pca__n_components': 1}


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:   42.2s finished


In [21]:
scaler = StandardScaler()

pca = PCA()

knc = KNeighborsClassifier()

step = [('scaler', scaler),('pca', pca),("KNeighborsClassifier", knc)]
pipe = Pipeline(step)
param_grid = {"KNeighborsClassifier__n_neighbors":list(range(1500,2501,250)), "pca__n_components":list(range(1,12,2))}
grid_search = GridSearchCV(pipe, param_grid, cv = 10, verbose = 1, scoring = 'accuracy')

grid_search.fit(X_train, y_train_cl)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.5715754833401893
{'KNeighborsClassifier__n_neighbors': 2500, 'pca__n_components': 3}


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  1.5min finished


In [22]:
scaler = StandardScaler()

pca = PCA()

knc = KNeighborsClassifier()

step = [('scaler', scaler),('pca', pca),("KNeighborsClassifier", knc)]
pipe = Pipeline(step)
param_grid = {"KNeighborsClassifier__n_neighbors":list(range(2250,3251,250)), "pca__n_components":list(range(1,6))}
grid_search = GridSearchCV(pipe, param_grid, cv = 10, verbose = 1, scoring = 'accuracy')

grid_search.fit(X_train, y_train_cl)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.5771287535993418
{'KNeighborsClassifier__n_neighbors': 3250, 'pca__n_components': 5}


[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:  1.3min finished


In [23]:
scaler = StandardScaler()

pca = PCA()

knc = KNeighborsClassifier()

step = [('scaler', scaler),('pca', pca),("KNeighborsClassifier", knc)]
pipe = Pipeline(step)
param_grid = {"KNeighborsClassifier__n_neighbors":list(range(3000,3501,100)), "pca__n_components":list(range(2,8))}
grid_search = GridSearchCV(pipe, param_grid, cv = 10, verbose = 1, scoring = 'accuracy')

grid_search.fit(X_train, y_train_cl)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.5785684903331962
{'KNeighborsClassifier__n_neighbors': 3300, 'pca__n_components': 5}


[Parallel(n_jobs=1)]: Done 360 out of 360 | elapsed:  2.2min finished


In [24]:
scaler = StandardScaler()

pca = PCA()

knc = KNeighborsClassifier()

step = [('scaler', scaler),('pca', pca),("KNeighborsClassifier", knc)]
pipe = Pipeline(step)
param_grid = {"KNeighborsClassifier__n_neighbors":list(range(3200,3400,5)), "pca__n_components":list(range(4,7))}
grid_search = GridSearchCV(pipe, param_grid, cv = 100, verbose = 1, scoring = 'accuracy')

grid_search.fit(X_train, y_train_cl)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 100 folds for each of 120 candidates, totalling 12000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.5682846565199506
{'KNeighborsClassifier__n_neighbors': 3275, 'pca__n_components': 4}


[Parallel(n_jobs=1)]: Done 12000 out of 12000 | elapsed: 17.5min finished
C:\Users\jacob\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
